In [1]:
import json
import pennylane as qml
from pennylane import numpy as np

In [23]:
# Uneditable section #
def create_Hamiltonian(h):
    """
    Function in charge of generating the Hamiltonian of the statement.

    Args:
        h (float): magnetic field strength

    Returns:
        (qml.Hamiltonian): Hamiltonian of the statement associated to h
    """
    # End of uneditable section #

    # Put your code here #
    num_qubits = 4
    
    
    coeffs1 = num_qubits*[-1]
    obs1 = [qml.PauliZ(wire) @ qml.PauliZ((wire + 1) % num_qubits) for wire in range(num_qubits)]
    
    coeffs2 = num_qubits*[-h]
    obs2 = [qml.PauliX(wire) for wire in range(num_qubits)]
    
    coeffs = coeffs1 + coeffs2
    obs = obs1 + obs2
    
    return qml.Hamiltonian(coeffs, obs)


# Uneditable section #
dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev)
def model(params, H):
    """
    To implement VQE you need an ansatz for the candidate ground state!
    Define here the VQE ansatz in terms of some parameters (params) that
    create the candidate ground state. These parameters will
    be optimized later.

    Args:
        params (numpy.array): parameters to be used in the variational circuit
        H (qml.Hamiltonian): Hamiltonian used to calculate the expected value

    Returns:
        (float): Expected value with respect to the Hamiltonian H
    """

    # End of uneditable section #

    # Put your code here #
    qml.BasicEntanglerLayers(weights=params[0], wires=range(4), rotation=qml.RX)
    qml.BasicEntanglerLayers(weights=params[1], wires=range(4), rotation=qml.RY)
    qml.BasicEntanglerLayers(weights=params[2], wires=range(4), rotation=qml.RZ)
    
    return qml.expval(H)
    


# Uneditable section #
def train(h):
    """
    In this function you must design a subroutine that returns the
    parameters that best approximate the ground state.

    Args:
        h (float): magnetic field strength

    Returns:
        (numpy.array): parameters that best approximate the ground state.
    """

    # End of uneditable section #

    # Put your code here #
    opt=qml.GradientDescentOptimizer(stepsize=0.1)
    max_iterations = 1000
    conv_tol = 1e-4
    
    Hamiltonian = create_Hamiltonian(h)
    params = np.random.rand(3,3,4, requires_grad=True)
    
    for n in range(max_iterations):
        params, prev_cost = opt.step_and_cost(model, params, H=Hamiltonian)
        new_cost = model(params, Hamiltonian)
        
        
    
    return params

In [24]:
def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    params = train(ins)
    return str(model(params, create_Hamiltonian(ins)))


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-1
    ), "The expected value is not correct."

In [25]:
test_cases = [['1.0', '-5.226251859505506'], ['2.3', '-9.66382463698038'], ['0.5', '-4.271558410139714'], ['1.5', '-6.760008550556145']]

In [ ]:
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '1.0'...
Correct!
Running test case 1 with input '2.3'...
